In [2]:
# base libraries
import numpy as np
import pandas as pd
import regex as re
import itertools
import sklearn.metrics as sk
from functools import reduce

# deep learning libraries
import torch
import transformers
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs

# hyperparameter optimization
import optuna
from optuna.samplers import TPESampler
import optuna.visualization.matplotlib as oviz

# file system manipulation
import os
import shutil
from pathlib import Path
import pickle
import copy

# logging
import logging
import time

In [5]:
# set seeds to make computations deterministic
np.random.seed(42)
torch.manual_seed(42)

# check CUDA availability
cuda_available = torch.cuda.is_available()
print("Is CUDA available? ", "Yes" if cuda_available else "No")

Is CUDA available?  Yes


In [6]:
final_test = pd.read_csv(r"Storage/Bert/test_8_11.csv")
final_test.columns = ["PatientID", "text", "labels"]

In [7]:
best_model = ClassificationModel(
    "bert",
    "Storage/Bert/NoHyperParameterTuningResults/trial_0"
)

In [8]:
test_results, test_outputs, test_wrong = best_model.eval_model(
    final_test,
)

  0%|          | 0/293 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/74 [00:00<?, ?it/s]

/space/mindds/2/projects/apoe-dl/apoe_slat/apoe-dl-env/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [9]:
max_prob_list = []
test_prob_list = []
test_pred_list = []
for i in range(len(final_test)):
    # prob_list = list(torch.softmax(torch.from_numpy(model_outputs[i]), axis=0)[:,1])
    prob_list = torch.softmax(torch.from_numpy(test_outputs[i]), axis=0)
    #print("Prob List: ", prob_list, type(prob_list))

    extracted_prob_list = []
    for i in range(len(prob_list)):
        extracted_prob_list.append(float(prob_list[i]))

    #print("Extracted Prob List: ", extracted_prob_list)
    # find max one in each submatrix of length 3
    max_proba = max(extracted_prob_list)

    # identify model prediction based on location of max_proba within extracted_prob_list
    if (extracted_prob_list[0] == max_proba):
        test_pred_list.append(0)
    elif (extracted_prob_list[1] == max_proba):
        test_pred_list.append(1)
    else:
        test_pred_list.append(2)

    max_prob_list.append(max_proba)
    test_prob_list.append(extracted_prob_list)

In [10]:
final_test["pred"] = test_pred_list
final_test["proba"] = max_prob_list

In [12]:
revised_preds = pd.read_csv(r"Storage/Bert/Revised Bert Preds.csv")

In [13]:
edits_made = 0
for i in range(len(final_test)):
    for j in range(len(revised_preds)):
        if (final_test.at[i, "text"] == revised_preds.at[j, "text"]):
            edits_made += 1
            final_test.at[i, "labels"] = revised_preds.at[j, "labels"]

In [15]:
sk.accuracy_score(test_pred_list, final_test["labels"])

0.9283276450511946

In [16]:
final_test.head()

,PatientID,text,labels,pred,proba
0,Z9096104,ital status: married civil union spouse name: ...,1,1,0.999421
1,Z7410349,history narrative social history tobacco quit ...,1,1,0.999419
2,Z7754903,ncy-based suicidal statements. suicidal homici...,0,0,0.999304
3,Z11959548,js; bjshingleton li61ao +21.5d target plano • ...,1,1,0.999390
4,Z7580275,rts pedal edema is worsened recently. current ...,2,2,0.997088


In [17]:
sk.roc_auc_score(final_test["labels"], test_prob_list, multi_class = "ovr", average = "weighted")

0.9764382581332275

In [18]:
import pickle

f = open("Storage/Bert/test_set_prob.pkl", "wb")
pickle.dump(str(test_prob_list), f)
f.close()

In [23]:
from sklearn import metrics

target_names = ['Negative', 'Neither', 'Positive']
results = metrics.classification_report(final_test["labels"], final_test["pred"], target_names = target_names, output_dict=True)
results = pd.DataFrame(results).transpose()
conf_mat = metrics.confusion_matrix(final_test["labels"], final_test["pred"])
print("Micro F1: ", metrics.f1_score(final_test["labels"], final_test["pred"], average = "micro"))
print("Macro F1: ", metrics.f1_score(final_test["labels"], final_test["pred"], average = "macro"))
print("Weighted F1: ", metrics.f1_score(final_test["labels"], final_test["pred"], average = "weighted"))
print("Confusion Matrix: \n", conf_mat)
print("Classification Report: \n", results)

Micro F1:  0.9283276450511946
Macro F1:  0.9174209846839186
Weighted F1:  0.9277437001702764
Confusion Matrix: 
 [[102   1   1]
 [  7 125   2]
 [  6   4  45]]
Classification Report: 
               precision    recall  f1-score     support
Negative       0.886957  0.980769  0.931507  104.000000
Neither        0.961538  0.932836  0.946970  134.000000
Positive       0.937500  0.818182  0.873786   55.000000
accuracy       0.928328  0.928328  0.928328    0.928328
macro avg      0.928665  0.910596  0.917421  293.000000
weighted avg   0.930553  0.928328  0.927744  293.000000


In [24]:
FP = conf_mat.sum(axis = 0) - np.diag(conf_mat) 
FN = conf_mat.sum(axis = 1) - np.diag(conf_mat)
TP = np.diag(conf_mat)
TN = conf_mat.sum() - (FP + FN + TP)
FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)

# Specificity or true negative rate
TNR = TN/(TN+FP) 

# Precision or positive predictive value
PPV = TP/(TP+FP)

# Negative predictive value
NPV = TN/(TN+FN)

# Fall out or false positive rate
FPR = FP/(FP+TN)

# False negative rate
FNR = FN/(TP+FN)

# False discovery rate
FDR = FP/(TP+FP)

print("Sensitivity: ", TPR)
print("Specificity: ", TNR)
print("NPV: ", NPV)
print("PPV: ", PPV)
print("FPR: ", FPR)

Sensitivity:  [0.98076923 0.93283582 0.81818182]
Specificity:  [0.93121693 0.96855346 0.98739496]
NPV:  [0.98876404 0.94478528 0.95918367]
PPV:  [0.88695652 0.96153846 0.9375    ]
FPR:  [0.06878307 0.03144654 0.01260504]


In [27]:
(0.93121693 +  0.96855346 + 0.98739496) / 3

0.9623884500000001

In [26]:
(0.98076923 + 0.93283582 + 0.81818182) / 3

0.9105956233333333